In [147]:
import requests
import bs4
import os
import re
import unicodedata
import pandas as pd
import nltk

import selenium

In [148]:
url = ('https://www.newegg.com/p/pl?N=100007709%20601361654%20601359415')
response = requests.get(url)
html = response.text
soup = bs4.BeautifulSoup(html)

In [149]:
response

<Response [200]>

In [150]:
soup

<!DOCTYPE html>
<html lang="en-us"><head><title>GeForce RTX 3060,GeForce RTX 3060 Ti Desktop Graphics Cards | Newegg.com</title><meta charset="utf-8"/><meta content="text/html; charset=utf-8" http-equiv="content-type"/><meta content="always" name="referrer"/><meta content="GeForce RTX 3060 Desktop Graphics Cards,GeForce RTX 3060 Ti Desktop Graphics Cards" name="keywords"/><meta content="Shop GeForce RTX 3060,GeForce RTX 3060 Ti Desktop Graphics Cards on Newegg.com. Watch for amazing deals and get great pricing." name="description"/><meta content="https://c1.neweggimages.com/WebResource/Themes/Nest/logos/logo_newegg_400400.png" property="og:image"/><meta content="Shop GeForce RTX 3060,GeForce RTX 3060 Ti Desktop Graphics Cards on Newegg.com. Watch for amazing deals and get great pricing." property="og:description"/><meta content="https://www.newegg.com/p/pl?N=100007709%20601359415" property="og:url"/><meta content="website" property="og:type"/><meta content="GeForce RTX 3060,GeForce RTX

In [151]:
items = soup.find_all('div', {'class': 'item-cell'})
items

[<div class="item-cell" id="item_cell_9SIAFVFFE77525_1_0"><div class="item-container"><a class="item-img" href="https://www.newegg.com/zotac-geforce-rtx-3060-ti-zt-a30610h-10mlhr/p/N82E16814500518"><img alt="ZOTAC GAMING GeForce RTX 3060 Ti Twin Edge OC LHR 8GB GDDR6 256-bit 14 Gbps PCIE 4.0 Gaming Graphics Card, IceStorm 2.0 Advanced Cooling, Active Fan Control, FREEZE Fan Stop ZT-A30610H-10MLHR" src="https://c1.neweggimages.com/ProductImageCompressAll300/14-500-518-01.jpg" title="ZOTAC GAMING GeForce RTX 3060 Ti Twin Edge OC LHR 8GB GDDR6 256-bit 14 Gbps PCIE 4.0 Gaming Graphics Card, IceStorm 2.0 Advanced Cooling, Active Fan Control, FREEZE Fan Stop ZT-A30610H-10MLHR"/></a><div class="item-info"><div class="item-branding"><a class="item-brand" href="https://www.newegg.com/ZOTAC/BrandStore/ID-12150"><img alt="ZOTAC" src="https://c1.neweggimages.com/Brandimage_70x28/Brand12150.gif" title="ZOTAC"/></a><a class="item-rating" href="https://www.newegg.com/zotac-geforce-rtx-3060-ti-zt-a306

In [152]:
soup.find('a', {'class':'item-title'}).text

'ZOTAC GAMING GeForce RTX 3060 Ti Twin Edge OC LHR 8GB GDDR6 256-bit 14 Gbps PCIE 4.0 Gaming Graphics Card, IceStorm 2.0 Advanced Cooling, Active Fan Control, FREEZE Fan Stop ZT-A30610H-10MLHR'

In [153]:
soup.find('li', {'class': 'price-was'}).text

'$849.67'

In [154]:
soup.find('li', {'class': 'price-current'}).text

'$809.67\xa0(4 Offers)–'

In [155]:
soup.find('p', {'class': "item-promo"}).text

AttributeError: 'NoneType' object has no attribute 'text'

In [146]:
soup.find('a.href').text

AttributeError: 'NoneType' object has no attribute 'text'

In [167]:
df = pd.DataFrame()
count=0
for item in items:
    count+=1
    item_title = item.find('a', {'class':'item-title'}).text
    old_price = item.find('li', {'class': 'price-was'}).text
    current_price = item.find('li', {'class': 'price-current'}).text
    
    print(item_title, '\n')

ZOTAC GAMING GeForce RTX 3060 Ti Twin Edge OC LHR 8GB GDDR6 256-bit 14 Gbps PCIE 4.0 Gaming Graphics Card, IceStorm 2.0 Advanced Cooling, Active Fan Control, FREEZE Fan Stop ZT-A30610H-10MLHR 

GIGABYTE AORUS GeForce RTX 3060 12GB GDDR6 PCI Express 4.0 ATX Video Card GV-N3060AORUS E-12GD (rev. 2.0) 

GIGABYTE Eagle GeForce RTX 3060 Ti 8GB GDDR6 PCI Express 4.0 ATX Video Card GV-N306TEAGLE OC-8GD (rev. 2.0) 

MSI Ventus GeForce RTX 3060 Ti 8GB GDDR6 PCI Express 4.0 Video Card RTX 3060 Ti VENTUS 2X 8G OCV1 LHR 

ASUS ROG Strix GeForce RTX 3060 12GB GDDR6 PCI Express 4.0 Video Card ROG-STRIX-RTX3060-O12G-V2-GAMING 

ASUS Dual GeForce RTX 3060 12GB GDDR6 PCI Express 4.0 Video Card DUAL-RTX3060-O12G-V2 

GIGABYTE Gaming GeForce RTX 3060 Ti 8GB GDDR6 PCI Express 4.0 ATX Video Card GV-N306TGAMINGOC PRO-8GD (rev. 3.0) 

ASUS TUF Gaming GeForce RTX 3060 12GB GDDR6 PCI Express 4.0 Video Card TUF-RTX3060-O12G-V2-GAMING 

ASUS TUF Gaming GeForce RTX 3060 Ti 8GB GDDR6 PCI Express 4.0 Video Card TUF